In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import matplotlib.pyplot as plt
import matplotlib
import cv2

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.utils import to_categorical, Sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop,Adam
#from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#from sklearn.metrics import classification_repo

In [6]:
path = '/kaggle/input/hpa-single-cell-image-classification/'
#os.listdir(path)

In [7]:
data = pd.read_csv(path+'train.csv')
ids = ["../input/hpa-single-cell-image-classification/train/" + fname + '_green.png' for fname in data['ID']]
num_classes = 19
def get_labels(labels):
    new_labels = []
    for label in labels:
        label = label.split('|')
        label = list(map(int, label))
        label = to_categorical(label, num_classes=num_classes)
        label = label.sum(axis=0)
        new_labels.append(label)
    return new_labels
labels = get_labels(data['Label'])
num_images = 21806

In [8]:
data['Label']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(ids[:num_images],labels[:num_images], 
                                                      test_size = 0.2)
X_train, X_test, y_train, y_test = train_test_split(ids[:num_images],labels[:num_images], 
                                                      test_size = 0.1)



In [11]:
len(y_test)

In [12]:
#X_test[:2],y_test[:2]

In [13]:
IMG_SIZE = 64

def process_image(image_path, img_size = IMG_SIZE):
  """
  Takes an image file path and turns the image into a Tensor. 
  """
  # Read in an image file
  image = tf.io.read_file(image_path)
  # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
  image = tf.image.decode_jpeg(image, channels = 3)
  # Convert the colour channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image, tf.float32)
  # Resize the image to our desired value (224, 224)
  image = tf.image.resize(image,size = [img_size, img_size])
  return image

In [15]:
def get_image_label(image_path, label):
  """
  Takes an image file path name and the assosciated label,
  processes the image and reutrns a typle of (image, label).
  """
  image = process_image(image_path)
  return image,label

In [16]:

BATCH_SIZE = 64

def create_data_batches(X , y = None, batch_size = BATCH_SIZE, valid_data = False, test_data = False):
  """
  Creates batches of data out of image (X) and label (y) pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's validation data.
  Also accepts test data as input (no labels).
  """
  if test_data:
    print('Creating test data batches........')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    data_batch = data.map(get_image_label).batch(batch_size)
    return data_batch
  
  elif valid_data:
    print('Creating valid data batches...........')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    data_batch = data.map(get_image_label).batch(batch_size)
    return data_batch
  
  else:
    print('Creating training data batches...............')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),tf.constant(y)))
    data = data.shuffle(buffer_size = len(X))
    data_batch = data.map(get_image_label).batch(batch_size)
    return data_batch

In [17]:

train_data = create_data_batches(X_train,y_train)
valid_data = create_data_batches(X_valid,y_valid, valid_data=True)
test_data = create_data_batches(X_test,y_test, test_data=True)


In [18]:
img_size = [64, 64]
img_channel = 3

In [19]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
import wandb
base_model = DenseNet121(include_top=False, weights='imagenet')
base_model.trainable = True

inputs = layers.Input((img_size[0],img_size[0], 3))

x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(19, activation='sigmoid')(x)

tf.keras.backend.clear_session()

model = Model(inputs, outputs)
model.summary()

In [20]:

#model.compile(optimizer=Adam(lr=learning_rate), loss="binary_crossentropy", metrics='accuracy')
model.compile(optimizer=Adam(learning_rate=0.1),loss='categorical_crossentropy',metrics='accuracy')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience=3)

In [21]:
model.fit(train_data,validation_data=valid_data,epochs=1)

In [22]:
print("on test data")
y_pred=model.evaluate(test_data)
print("accuaracy", str(y_pred[1]*100))
print("Total loss",str(y_pred[0]*100))

In [50]:
y_pred2=model.predict(test_data)
args1 = np.argmax(np.array(y_test), axis=1)
args2 = np.argmax(y_pred2, axis=1)


In [53]:
from sklearn.metrics import accuracy_score
print("accuracy : ",accuracy_score(args1, args2))

In [60]:
from sklearn.metrics import classification_report
#print(classification_report(np.array(y_test), y_pred2))
print(classification_report(args1, args2))

In [ ]:
print("on train data")
y_train_pred=model.evaluate(train_data)
print("accuaracy", str(y_train_pred[1]*100))
print("Total loss",str(y_train_pred[0]*100))

In [ ]:
y_train_pred2=model.predict(test_data)
train_args1 = np.argmax(np.array(y_test), axis=1)
train_args2 = np.argmax(y_train_pred2, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
print("train accuracy : ",accuracy_score(train_args1, train_args2))

In [ ]:
from sklearn.metrics import classification_report
#print(classification_report(np.array(y_test), y_pred2))
print(classification_report(train_args1, train_args2))